In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel('concordance_preloaded_notenten23_tt1_20250617165050.xlsx')

In [2]:
df

,Reference,Sentence
0,"ht.no,2021-01-12 04:34",<s> Harstad-jenta har gått på Heggen videregåe...
1,"tu.no,2020-12-28 22:10","<s> Det vil si, de kan også gå på diesel . </s>"
2,"tu.no,2020-12-30 09:20",<s> De resterende nesten tusen bussene går på ...
3,"vi.no,2020-12-24 20:51",<s> I artikkelen forteller Elsa hvordan hun gj...
4,"ks.no,2020-12-22 12:03",<s> Når vi nå åpner opp for de resterende komm...
...,...,...
9995,"blogspot.com,2020-12-27 09:41",<s> Ingrid lagde Chili con carne med utgangspu...
9996,"blogspot.com,2020-12-24 11:09","<s> Noen går på terrassen her, noen på terrass..."
9997,"norwegianforum.net,2020-12-23 11:02","<s> Hun har nå gått på antibiotika i 14 dager,..."
9998,"norwegianforum.net,2020-12-24 12:28",<s> Når mine går på tredemølle bruker jeg førs...


In [3]:
df[['URL', 'Date']] = df['Reference'].str.split(',',expand=True).copy()

In [4]:
df = df.drop(columns=['Reference'])

In [5]:
import re
# Define the function to normalize text
def normalize_text(text):
    """text cleaning just for whitespaces and amc sentence markers"""
    # remove <s>
    text = re.sub(r'<s>', '', text)
    text = re.sub(r'</s>', '', text)
    text = text.strip() # remove leading and trailing whitespace
    return text

In [6]:
df['text'] = df['Sentence'].apply(normalize_text)

In [7]:
!python -m spacy download nb_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.5 MB ? eta -:--:--
     -------------------------------- ------ 10.5/12.5 MB 50.4 MB/s eta 0:00:01
     --------------------------------------- 12.5/12.5 MB 35.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('nb_core_news_sm')



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import spacy
nlp = spacy.load("nb_core_news_sm")

In [18]:
def get_nouns_after_ga_pa(text):
    doc = nlp(text)
    nouns = []
    
    for i in range(len(doc)-2):  # -2 because we need at least 3 tokens (gå + på + noun)
        if doc[i].lemma_ == 'gå' and doc[i+1].text.lower() == 'på':
            # Get the next token and check if it's a noun
            j = i + 2
            while j < len(doc) and (doc[j].is_punct or doc[j].text.lower() in ['og', 'eller']):
                j += 1
            if j < len(doc) and doc[j].pos_ == 'NOUN':
                nouns.append(doc[j].text)
    
    # Join the nouns with comma if there are multiple nouns
    return ', '.join(nouns) if nouns else ''

# Apply the function to the 'text' column and create a new column 'nouns_after_ga_pa'
df['nouns'] = df['text'].apply(get_nouns_after_ga_pa)

In [20]:
df.count()

Sentence    10000
URL         10000
Date        10000
text        10000
nouns       10000
dtype: int64

In [22]:
df.to_excel('gå_på_nouns.xlsx', index=False)